# Matrices

For this exercise of seeing of numerical linear algebra in Python, we will be using a combination of numpy and scipy.
Scipy will be used in most cases but numpy will be relied upon 

## Matrix Operations



In [2]:
import numpy as np
import scipy.linalg as la
import scipy
import time
import sys


Creating an Array

In [3]:
# row vector
c = np.array([1,2])
print("row vector", c)
# column vector
d = np.array([[1],[2]])
print("column vector",d)

row vector [1 2]
column vector [[1]
 [2]]


Matrix Addition

In [4]:

# 2x2 matrix
a = np.array([[1,2],[3,4]])
b= np.array([[5,6],[7,8]])
print("matrix addition",a+b)


matrix addition [[ 6  8]
 [10 12]]


Matrix Multiplication

@ is typically more effiecient since it uses matmul underneath so it is the preferred method and was designed specifically for matrix multiplication. This will break if the number of columns in the first matrix is not equal to the number of rows in the second matrix.

In [5]:
multiply = np.dot(a,b)
print("matrix multiplication",multiply)
multiply_2 = a@b
print("matrix multiplication",multiply_2)

matrix multiplication [[19 22]
 [43 50]]
matrix multiplication [[19 22]
 [43 50]]


Sparse vs Dense Matrices

If we look at the matrix above we can see that it is automatically assumed that the matrix is **dense** which means that each number in the matrix is explicitly stores in memory. However, there may be a case where the matrix has a special form like a diagonal or upper triangular or lower triangular matrix. In this case, the only values that matter are the the non-zero values, so ideally we only want to store those in memory to speed up computation and reduce memory usage. This is where **sparse** matrices come in.

By default all matrices created in numpy are dense but if you want to create a sparse matrix you can use scipy's linalg module. 

In [6]:
# Creating a very large matrix
N = 10000
num_zeros = int(N * N * 0.9)  # Correctly calculating 90% of total elements

matrix = np.random.rand(N, N)
flat_matrix = matrix.flatten()
zero_indices = np.random.choice(flat_matrix.size, num_zeros, replace=False)

flat_matrix[zero_indices] = 0
matrix_with_zeros = flat_matrix.reshape(N, N)

dense_matrix = matrix_with_zeros
sparse_matrix = scipy.sparse.csr_matrix(matrix_with_zeros)  # Creating sparse matrix
random_vector = np.random.rand(N)

# Timing the dense matrix multiplication
start_dense = time.time()
result_dense = dense_matrix @ random_vector
end_dense = time.time()
print("Dense time:", end_dense - start_dense)

# Timing the sparse matrix multiplication
start_sparse = time.time()
result_sparse = sparse_matrix @ random_vector  # Using sparse matrix
end_sparse = time.time()
print("Sparse time:", end_sparse - start_sparse)

# Memory usage
dense_memory = dense_matrix.nbytes
sparse_memory = sparse_matrix.data.nbytes + sparse_matrix.indptr.nbytes + sparse_matrix.indices.nbytes

print(f"Dense Matrix Memory: {dense_memory / (1024 ** 2):.2f} MB")
print(f"Sparse Matrix Memory: {sparse_memory / (1024 ** 2):.2f} MB")


Dense time: 0.07150435447692871
Sparse time: 0.011000871658325195
Dense Matrix Memory: 762.94 MB
Sparse Matrix Memory: 114.48 MB


Dot Product Algorithm


In [8]:
def dot_product(x, y):
    c=0
    for i in range(len(x)):
        c+= x[i]*y[i]
    return c

x = np.random.rand(5)
y = np.random.rand(5)
z= dot_product(x,y)
print(x,y,z)

[0.396859   0.88535457 0.06062314 0.40088156 0.4839757 ] [0.37107676 0.34731111 0.86061728 0.71736089 0.33114384] 0.9547742663614411


Forward Substitution

Refer to the Notes for the details regarding the Algorithm

In [15]:
def forward_substitution_row_oriented(L, b):
    """
    Row-oriented forward substitution for solving Lx = b.
    This algorithm overwrites b with the solution x.
    Parameters:
        L: Lower triangular matrix (n x n, numpy array)
        b: Right-hand side vector (n, numpy array), will be overwritten with the solution
    Returns:
        b: Solution vector (overwrites the input b)
    """
    n = len(b)

    # Solve for the first variable
    b[0] = b[0] / L[0, 0]

    # Solve for the remaining variables
    for i in range(1, n):
        b[i] = (b[i] - np.dot(L[i, :i], b[:i])) / L[i, i]

    return b

In [21]:
# Define a lower triangular matrix L and vector b
L = np.array([
    [2, 0, 0],
    [3, 1, 0],
    [1, -1, 1]
], dtype=float)

b = np.array([4, 5, 6], dtype=float)

# Use the function with a copy of b
x = forward_substitution_row_oriented(L, b.copy())
print("Solution vector x:", x)

# Verify original b remains unchanged
print("Original b vector:", b)

# Compare with SciPy
x_scipy = la.solve_triangular(L, b, lower=True)
print("Solution vector x (SciPy):", x_scipy)

Solution vector x: [ 2. -1.  3.]
Original b vector: [4. 5. 6.]
Solution vector x (SciPy): [ 2. -1.  3.]


Backward Substitution

In [17]:
def backward_substitution(U, y):
    """
    Solves Ux = y using backward substitution, overwriting y with the solution x.

    Parameters:
        U: Upper triangular matrix (n x n, numpy array)
        y: Right-hand side vector (n, numpy array), will be overwritten with the solution x

    Returns:
        y: Solution vector (overwrites the input y)
    """
    n = len(y)

    # Step 1: Solve for y(n)
    y[n-1] = y[n-1] / U[n-1, n-1]

    # Step 2: Solve for y(i) for i = n-1 to 1
    for i in range(n-2, -1, -1):  # Iterate from n-2 to 0
        sum_terms = np.dot(U[i, i+1:], y[i+1:])  # Compute sum of U[i, j] * y[j] for j = i+1 to n
        y[i] = (y[i] - sum_terms) / U[i, i]

    return y

In [18]:
# Define an upper triangular matrix U and right-hand side vector y
U = np.array([
[2, 1, -1],
[0, 3, 2],
[0, 0, 1]
], dtype=float)

y = np.array([5, 8, 3], dtype=float)

# Solve Ux = y using backward substitution
x = backward_substitution(U, y)
print("Solution vector x:", x)
x_backward = la.solve_triangular(U, b, lower=False)
print("Solution for backward substitution (Ux = b):", x_backward)

Solution vector x: [3.66666667 0.66666667 3.        ]
Solution for backward substitution (Ux = b): [ 3.66666667 -2.33333333  3.        ]


In [24]:
import numpy as np
from scipy.linalg import lu


def gaussian_elimination_no_pivoting(A, b):
    """
    Solves Ax = b using Gaussian elimination without pivoting.
    Demonstrates instability for poorly scaled systems.
    """
    n = len(A)
    U = A.copy()
    L = np.eye(n)  # Start with L as an identity matrix

    # Gaussian elimination
    for k in range(n - 1):
        if abs(U[k, k]) < 1e-12:
            raise ValueError(f"Zero (or near-zero) pivot encountered at U[{k}, {k}]!")
        for i in range(k + 1, n):
            # Compute the multiplier
            m = U[i, k] / U[k, k]
            L[i, k] = m  # Store the multiplier in L
            # Update row i of U
            U[i, :] -= m * U[k, :]

    # Solve Ly = b (forward substitution)
    y = np.zeros_like(b)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])

    # Solve Ux = y (back substitution)
    x = np.zeros_like(b)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]

    return x, L, U


def gaussian_elimination_with_pivoting(A, b):
    """
    Solves Ax = b using Gaussian elimination with partial pivoting.
    """
    n = len(A)
    U = A.copy()
    L = np.eye(n)  # Start with L as an identity matrix
    P = np.eye(n)  # Start with P as an identity matrix

    # Gaussian elimination with partial pivoting
    for k in range(n - 1):
        # Partial pivoting: Find the row with the largest pivot and swap rows
        max_row = np.argmax(abs(U[k:, k])) + k
        if max_row != k:
            U[[k, max_row]] = U[[max_row, k]]
            P[[k, max_row]] = P[[max_row, k]]
            if k > 0:
                L[[k, max_row], :k] = L[[max_row, k], :k]

        for i in range(k + 1, n):
            if abs(U[k, k]) < 1e-12:
                raise ValueError(f"Zero (or near-zero) pivot encountered at U[{k}, {k}]!")
            # Compute the multiplier
            m = U[i, k] / U[k, k]
            L[i, k] = m  # Store the multiplier in L
            # Update row i of U
            U[i, :] -= m * U[k, :]

    # Solve Ly = Pb (forward substitution)
    b_prime = np.dot(P, b)
    y = np.zeros_like(b)
    for i in range(n):
        y[i] = b_prime[i] - np.dot(L[i, :i], y[:i])

    # Solve Ux = y (back substitution)
    x = np.zeros_like(b)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]

    return x, P, L, U


# Define the matrix and vector
A = np.array([
    [0.0001, 1, 1],
    [1, 2, 3],
    [2, 4, 8]
], dtype=float)

b = np.array([2, 10, 20], dtype=float)

# Solve without pivoting
print("Gaussian Elimination Without Pivoting:")
try:
    x_no_pivot, L_no_pivot, U_no_pivot = gaussian_elimination_no_pivoting(A, b)
    print("Solution (x):", x_no_pivot)
    print("L (lower triangular):\n", L_no_pivot)
    print("U (upper triangular):\n", U_no_pivot)
except ValueError as e:
    print("Error:", e)

# Solve with partial pivoting
print("\nGaussian Elimination With Partial Pivoting:")
x_pivot, P_pivot, L_pivot, U_pivot = gaussian_elimination_with_pivoting(A, b)
print("Solution (x):", x_pivot)
print("P (permutation matrix):\n", P_pivot)
print("L (lower triangular):\n", L_pivot)
print("U (upper triangular):\n", U_pivot)



Gaussian Elimination Without Pivoting:
Solution (x): [6.00120024 1.99939988 0.        ]
L (lower triangular):
 [[1.e+00 0.e+00 0.e+00]
 [1.e+04 1.e+00 0.e+00]
 [2.e+04 2.e+00 1.e+00]]
U (upper triangular):
 [[ 1.000e-04  1.000e+00  1.000e+00]
 [ 0.000e+00 -9.998e+03 -9.997e+03]
 [ 0.000e+00  0.000e+00  2.000e+00]]

Gaussian Elimination With Partial Pivoting:
Solution (x): [ 6.00120024  1.99939988 -0.        ]
P (permutation matrix):
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
L (lower triangular):
 [[1.e+00 0.e+00 0.e+00]
 [5.e-05 1.e+00 0.e+00]
 [5.e-01 0.e+00 1.e+00]]
U (upper triangular):
 [[ 2.      4.      8.    ]
 [ 0.      0.9998  0.9996]
 [ 0.      0.     -1.    ]]


In [25]:
import numpy as np

# Define L and U from the LU decomposition
L = np.array([
    [1, 0, 0],
    [2, 1, 0],
    [3, 9, 1]
], dtype=float)

U = np.array([
    [2, 3, 1],
    [0, 1, 1],
    [0, 0, -7]
], dtype=float)

# Perform LU multiplication
LU = np.dot(L, U)

# Define the original matrix A for comparison
A = np.array([
    [2, 3, 1],
    [4, 7, 3],
    [6, 18, 5]
], dtype=float)

# Print results
print("Matrix L:")
print(L)
print("\nMatrix U:")
print(U)
print("\nMatrix LU (L multiplied by U):")
print(LU)
print("\nOriginal Matrix A:")
print(A)

# Verify if LU equals A
if np.allclose(LU, A):
    print("\nLU equals A: Verification successful!")
else:
    print("\nLU does not equal A: Verification failed.")


Matrix L:
[[1. 0. 0.]
 [2. 1. 0.]
 [3. 9. 1.]]

Matrix U:
[[ 2.  3.  1.]
 [ 0.  1.  1.]
 [ 0.  0. -7.]]

Matrix LU (L multiplied by U):
[[ 2.  3.  1.]
 [ 4.  7.  3.]
 [ 6. 18.  5.]]

Original Matrix A:
[[ 2.  3.  1.]
 [ 4.  7.  3.]
 [ 6. 18.  5.]]

LU equals A: Verification successful!


In [28]:
import numpy as np

# Define matrix A and vector b
A = np.array([
    [0.001, 1.0],
    [1.0, 2.0]
], dtype=float)

b = np.array([1.0, 3.0], dtype=float)

# Custom Gaussian elimination with rounding
def gaussian_elimination_with_rounding(A, b, precision=3):
    n = len(A)
    U = A.copy()
    L = np.eye(n)
    
    # Gaussian elimination with rounding
    for k in range(n - 1):
        for i in range(k + 1, n):
            m = round(U[i, k] / U[k, k], precision)
            L[i, k] = m
            U[i, :] -= m * U[k, :]
            U[i, :] = np.round(U[i, :], precision)  # Round U to simulate lower precision
    
    # Forward substitution (rounding)
    y = np.zeros_like(b)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
        y[i] = round(y[i], precision)
    
    # Backward substitution (rounding)
    x = np.zeros_like(b)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]
        x[i] = round(x[i], precision)
    
    return x, L, U

# Solve the system
computed_x, L, U = gaussian_elimination_with_rounding(A, b, precision=3)

# Print results
print("Computed Solution (x̂):", computed_x)
print("Matrix L (Lower Triangular):")
print(L)
print("Matrix U (Upper Triangular):")
print(U)




Computed Solution (x̂): [1.    0.999]
Matrix L (Lower Triangular):
[[   1.    0.]
 [1000.    1.]]
Matrix U (Upper Triangular):
[[ 1.00e-03  1.00e+00]
 [ 0.00e+00 -9.98e+02]]


In [1]:

import numpy as np
import functools

def count_flops(func):
    """
    Decorator to count the number of floating-point operations (FLOPs) 
    performed in a function.
    """
    # Dictionary to hold FLOP counts
    flop_counter = {"count": 0}

    # Helper to count FLOPs for numpy functions
    def count_operations(op_name, *args, **kwargs):
        if op_name == "matmul":  # Matrix multiplication
            # Assuming the operation is np.matmul(a, b)
            a, b = args[0], args[1]
            flop_counter["count"] += 2 * a.shape[0] * a.shape[1] * b.shape[1]
        elif op_name == "dot":  # Dot product
            a, b = args[0], args[1]
            flop_counter["count"] += 2 * a.size
        elif op_name == "add":  # Addition
            a, b = args[0], args[1]
            flop_counter["count"] += a.size
        elif op_name == "subtract":  # Subtraction
            a, b = args[0], args[1]
            flop_counter["count"] += a.size
        elif op_name == "multiply":  # Element-wise multiplication
            a, b = args[0], args[1]
            flop_counter["count"] += a.size
        elif op_name == "divide":  # Element-wise division
            a, b = args[0], args[1]
            flop_counter["count"] += a.size

    # Wrapping numpy functions
    original_matmul = np.matmul
    original_dot = np.dot
    original_add = np.add
    original_subtract = np.subtract
    original_multiply = np.multiply
    original_divide = np.divide

    def wrap_and_count(op, op_name):
        def wrapped(*args, **kwargs):
            count_operations(op_name, *args, **kwargs)
            return op(*args, **kwargs)
        return wrapped

    np.matmul = wrap_and_count(original_matmul, "matmul")
    np.dot = wrap_and_count(original_dot, "dot")
    np.add = wrap_and_count(original_add, "add")
    np.subtract = wrap_and_count(original_subtract, "subtract")
    np.multiply = wrap_and_count(original_multiply, "multiply")
    np.divide = wrap_and_count(original_divide, "divide")

    # Main decorator function
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Reset FLOP counter
        flop_counter["count"] = 0
        result = func(*args, **kwargs)
        print(f"FLOPs in {func.__name__}: {flop_counter['count']}")
        return result

    # Restore original numpy functions after the function call
    def restore_numpy():
        np.matmul = original_matmul
        np.dot = original_dot
        np.add = original_add
        np.subtract = original_subtract
        np.multiply = original_multiply
        np.divide = original_divide

    # Restore numpy functions on function exit
    @functools.wraps(func)
    def cleanup_wrapper(*args, **kwargs):
        try:
            return wrapper(*args, **kwargs)
        finally:
            restore_numpy()

    return cleanup_wrapper
@count_flops
def conjugate_gradient(A, b, x0=None, tol=1e-6, max_iter=1000):
    """
    Conjugate Gradient method for solving Ax = b.
    """
    n = len(b)
    if x0 is None:
        x0 = np.zeros_like(b)
    x = x0
    r = b - np.matmul(A, x)
    p = r
    rs_old = np.dot(r, r)
    
    for i in range(max_iter):
        Ap = np.matmul(A, p)
        alpha = rs_old / np.dot(p, Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        rs_new = np.dot(r, r)
        if np.sqrt(rs_new) < tol:
            break
        p = r + (rs_new / rs_old) * p
        rs_old = rs_new

    return x

# Example: Small Matrix
A = np.array([[4, 1], [1, 3]], dtype=float)
b = np.array([1, 2], dtype=float)

# Run Conjugate Gradient with FLOP counting
x = conjugate_gradient(A, b)


IndexError: tuple index out of range